https://cloud.google.com/storage/docs/reference/libraries#client-libraries-usage-python

In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage
import os
import pprint
import codecs
import json
import datetime

In [ ]:
root_path = "/Users/xszpo/GoogleDrive/01_Projects/201907_xFlats/"
gcp_secrets = "secrets/xflats-d81a8c5eaf54.json"
secters_path = os.path.join(root_path, gcp_secrets)

In [ ]:
storage_client = storage.Client.from_service_account_json(secters_path)

In [ ]:
# list buckets
list(storage_client.list_buckets())

In [ ]:
# The name for the new bucket
bucket_name = "xflats-666"

In [ ]:
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
a = bucket.blob('test.txt').upload_from_string('{"a":"test"}')

In [ ]:
blobs = storage_client.list_blobs(bucket_name)
for blob in blobs:
    print(blob.name)

In [ ]:
import json
import codecs

class Mongo2GCPstorage:
    
    def __init__(self, 
                 name, 
                 json_file_path, 
                 gcp_secret_path,
                 bucket_name = "xflats-666",
                 id_field_in_json='_id',
                 conv_download_date_utc = True, 
                 conv_download_date = True):
        self.name = name
        self.json_file_path = json_file_path
        self.gcp_secret_path = gcp_secret_path
        self.bucket_name = bucket_name
        self.id_field_in_json = id_field_in_json
        self.conv_download_date_utc = conv_download_date_utc
        self.conv_download_date = conv_download_date
        self.data = None
        self.bucket = None
        self.data = None

        self.storage_client = storage.Client.from_service_account_json(gcp_secret_path)

        if self.bucket_name in [i.name for i in self._list_gcp_buckets()]:
            print("Bucket {} has been found, ok.".format(self.bucket_name))
            self.bucketfound = True
        else:
            print("Bucket {} has not benn found...".format(self.bucket_name))
            self.bucketfound = False
        
        
    def _list_gcp_buckets(self):
        return list(self.storage_client.list_buckets())
        
    def _read_json_line(self):
        with codecs.open(self.json_file_path,'r') as file:
            data = file.readlines()
        return data
    
    def _convert_2_json(self, json_flat):
        json_data = json.loads(json_flat)
        if self.conv_download_date_utc:
            try:
                json_data['download_date_utc'] = float(json_data['download_date_utc']['$numberDecimal']) 
            except:
                json_data['download_date_utc'] = float(json_data['download_date_utc'])
        if self.conv_download_date :
            json_data['download_date'] = json_data['download_date']['$date']
        return json_data

    def _get_bucket(self):
        if self.bucketfound:
            self.bucket = self.storage_client.get_bucket(self.bucket_name)
        else:
            print('Bucker {} doesnt exist, couldnt process'.format(self.bucket_name))

    def file_to_json_line(self):
        data = self._read_json_line()
        data = [self._convert_2_json(i) for i in data]
        self.data = data
        
    def _save_json_to_gcp(self,one_json):
        self._get_bucket()
        self.bucket.blob(one_json[self.id_field_in_json]+".json").upload_from_string(json.dumps(one_json))
    
        
    def file_to_gcp(self):
        self.file_to_json_line()
        for one_json in self.data:
            self._save_json_to_gcp(one_json)
            break
        

In [ ]:
olx = Mongo2GCPstorage(
    name = "olx",
    id_prefix = "olx"
    json_file_path = '../mongodb/olx.json',
    gcp_secret_path = secters_path,
    bucket_name = "xflats-666"
)

In [ ]:
olx.file_to_json_line()

In [ ]:
olx.file_to_gcp()

In [ ]:
x=olx.bucket.get_blob('olx_316139968_430000.json')

In [ ]:
[i.name for i in olx.bucket.list_blobs()]

In [ ]:
olx.bucket.blob('olx_316139968_430000.json').exists()

In [ ]:
json.loads(x.download_as_string())

In [ ]:
gs://xflats-666/olx_316139968_430000.json

In [ ]:
olx.file_to_json_line()

x = olx.data

olx.file_to_gcp()
#datetime.datetime.utcfromtimestamp(1574498524.432)

In [ ]:
olx.get_bucket()

In [ ]:
json.dumps(x[0])